In [1]:
import os
import sys

import pandas as pd
import numpy as np
import geopandas as gpd
import folium

import matplotlib.pyplot as plt
import seaborn as sns

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/edgaregurrola/anaconda3/envs/odc/lib/python3.10/site-packages/geopandas/_compat.py:153: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  set_use_pygeos()
/tmp/ipykernel_29933/4027295472.py:6: DeprecationWarning: GeoPandas is set to use PyGEOS over Shapely. PyGEOS support is deprecatedand will be removed in GeoPandas 1.0, released in the Q1 of 2024. Please migrate to Shapely 2.0 (https://geopandas.org/en/stable/docs/user_guide/pygeos_to_shapely.html).
  import geopandas as gpd


2024-01-17 11:12:54 Configured OSMnx 1.1.2
2024-01-17 11:12:54 HTTP response caching is on


## Read top 10 cities

In [2]:
top_10 = pd.read_csv('../../../data/processed/nearshoring/top_10_cities_analysis.csv')

In [3]:
top_10.loc[0,"city"]

'CDMX'

## Test comparison process

In [30]:
schema = 'hexgrid'
table = 'hexgrid_8_city_2020'
query = f"SELECT * FROM {schema}.{table} WHERE \"city\" = \'{top_10.loc[0,'city']}\'"
hex_polygon = aup.gdf_from_query(query)

In [18]:
gdf = hex_polygon.to_crs("EPSG:6372")
gdf = gdf.buffer(1).reset_index().rename(columns={0: "geometry"})
gdf = gdf.set_geometry("geometry")
gdf = gdf.to_crs("EPSG:4326")
poly_wkt = gdf.dissolve().geometry.to_wkt()[0]

In [20]:
schema = 'censo'
table = 'censo_mza_centroid_2010'
query = f"SELECT * FROM {schema}.{table} WHERE ST_Intersects(geometry, 'SRID=4326;{poly_wkt}')"
censo_10 = aup.gdf_from_query(query, geometry_col="geometry")
print(censo_10.shape)
censo_10.head(4)

(63358, 193)


,cvegeo,geometry,pobtot,pobmas,pobfem,p_0a2,p_0a2_m,p_0a2_f,p_3ymas,p_3ymas_m,...,vph_radio,vph_tv,vph_refri,vph_lavad,vph_autom,vph_pc,vph_telef,vph_cel,vph_inter,metropolis
0,0900200011057004,POINT (-99.16472 19.46530),22,11,11,0,0,0,19,9,...,6,6,6,6,5,4,5,6,4,ZMVM
1,0900200010769046,POINT (-99.15564 19.46462),27,14,13,0,0,0,27,14,...,10,11,10,10,10,9,10,11,9,ZMVM
2,0900200010769045,POINT (-99.15569 19.46414),109,62,47,7,3,4,102,59,...,37,38,38,33,36,37,30,39,35,ZMVM
3,0900200011095024,POINT (-99.21222 19.48359),1194,580,614,49,26,23,1129,545,...,291,307,277,232,119,144,200,242,106,ZMVM


In [21]:
table = 'censo_mza_centroid_2020'
query = f"SELECT * FROM {schema}.{table} WHERE ST_Intersects(geometry, 'SRID=4326;{poly_wkt}')"
censo_20 = aup.gdf_from_query(query, geometry_col="geometry")
print(censo_20.shape)
censo_20.head(4)

(66559, 225)


,cvegeo,geometry,pobtot,pobfem,pobmas,p_0a2,p_0a2_f,p_0a2_m,p_3ymas,p_3ymas_f,...,vph_cel,vph_inter,vph_stvp,vph_spmvpi,vph_cvj,vph_sinrtv,vph_sinltc,vph_sincint,vph_sintic,metropolis
0,0900500010309001,POINT (-99.14573 19.52433),3,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,ZMVM
1,0900200011042049,POINT (-99.20754 19.47537),9,5,4,None,None,None,7,4,...,None,None,None,0,0,0,None,None,0,ZMVM
2,0900200011042048,POINT (-99.20757 19.47528),11,6,5,None,0,None,10,6,...,3,None,0,0,None,0,0,None,0,ZMVM
3,0900500013445037,POINT (-99.12610 19.57468),24,14,10,0,0,0,24,14,...,5,5,None,None,None,0,0,None,0,ZMVM


In [23]:
censo_20 = censo_20[['pobtot','vivtot','geometry']]
censo_20 = censo_20.rename(columns={'pobtot':'20_pobtot',
                                   'vivtot':'20_vivtot'})
censo_10 = censo_10[['pobtot','vivtot','geometry']]
censo_10 = censo_10.rename(columns={'pobtot':'10_pobtot',
                                   'vivtot':'10_vivtot'})

In [32]:
censo_hex = censo_20.overlay(hex_polygon, how='intersection')
censo_hex = censo_hex.groupby('hex_id_8').sum(numeric_only=True)
censo_hex = censo_hex.reset_index()
censo_hex

,hex_id_8,20_pobtot,20_vivtot
0,8849864921fffff,1930,673
1,8849864925fffff,855,339
2,8849864927fffff,289,93
3,8849864929fffff,85,35
4,884986492bfffff,664,206
...,...,...,...
1053,884995bb65fffff,16054,5293
1054,884995bb67fffff,11138,3840
1055,884995bb69fffff,7846,2345
1056,884995bb6bfffff,21531,6008


In [33]:
hex_hex_polygon.merge(censo_hex, on='hex_id_8', how='left')

,hex_id_8,geometry,CVEGEO,NOMGEO,city,type,20_pobtot,20_vivtot
0,884995b889fffff,"POLYGON ((-99.18300 19.49493, -99.17848 19.497...",09002,Azcapotzalco,CDMX,urban,9150.0,3004.0
1,884995b8d1fffff,"POLYGON ((-99.20183 19.49350, -99.19730 19.496...",09002,Azcapotzalco,CDMX,urban,12761.0,4018.0
2,884995b8e1fffff,"POLYGON ((-99.15894 19.48898, -99.15441 19.491...",09002,Azcapotzalco,CDMX,urban,12.0,2.0
3,884995b81bfffff,"POLYGON ((-99.17143 19.46475, -99.16690 19.467...",09002,Azcapotzalco,CDMX,urban,10234.0,4317.0
4,884995b8c9fffff,"POLYGON ((-99.18608 19.47140, -99.18156 19.474...",09002,Azcapotzalco,CDMX,urban,14614.0,5614.0
...,...,...,...,...,...,...,...,...
1762,8849958c91fffff,"POLYGON ((-99.05952 19.41885, -99.05499 19.421...",09017,Venustiano Carranza,CDMX,urban,18579.0,5954.0
1763,884995b959fffff,"POLYGON ((-99.10343 19.43883, -99.09891 19.441...",09017,Venustiano Carranza,CDMX,urban,17121.0,5529.0
1764,884995b94dfffff,"POLYGON ((-99.07415 19.42551, -99.06963 19.428...",09017,Venustiano Carranza,CDMX,urban,1670.0,465.0
1765,884995bb33fffff,"POLYGON ((-99.10760 19.43076, -99.10307 19.433...",09017,Venustiano Carranza,CDMX,urban,12975.0,4382.0


In [35]:
censo_hex = censo_20.overlay(hex_polygon, how='intersection')
censo_hex = censo_hex.groupby('hex_id_8').sum(numeric_only=True)
censo_hex = censo_hex.reset_index()
hex_pob = hex_polygon.merge(censo_hex, on='hex_id_8', how='left')

In [36]:
censo_hex = censo_10.overlay(hex_polygon, how='intersection')
censo_hex = censo_hex.groupby('hex_id_8').sum(numeric_only=True)
censo_hex = censo_hex.reset_index()
hex_pob = hex_pob.merge(censo_hex, on='hex_id_8', how='left')

In [37]:
hex_pob

,hex_id_8,geometry,CVEGEO,NOMGEO,city,type,20_pobtot,20_vivtot,10_pobtot,10_vivtot
0,884995b889fffff,"POLYGON ((-99.18300 19.49493, -99.17848 19.497...",09002,Azcapotzalco,CDMX,urban,9150.0,3004.0,11473.0,3515.0
1,884995b8d1fffff,"POLYGON ((-99.20183 19.49350, -99.19730 19.496...",09002,Azcapotzalco,CDMX,urban,12761.0,4018.0,12034.0,3661.0
2,884995b8e1fffff,"POLYGON ((-99.15894 19.48898, -99.15441 19.491...",09002,Azcapotzalco,CDMX,urban,12.0,2.0,178.0,50.0
3,884995b81bfffff,"POLYGON ((-99.17143 19.46475, -99.16690 19.467...",09002,Azcapotzalco,CDMX,urban,10234.0,4317.0,11747.0,4349.0
4,884995b8c9fffff,"POLYGON ((-99.18608 19.47140, -99.18156 19.474...",09002,Azcapotzalco,CDMX,urban,14614.0,5614.0,14470.0,5161.0
...,...,...,...,...,...,...,...,...,...,...
1762,8849958c91fffff,"POLYGON ((-99.05952 19.41885, -99.05499 19.421...",09017,Venustiano Carranza,CDMX,urban,18579.0,5954.0,20312.0,6204.0
1763,884995b959fffff,"POLYGON ((-99.10343 19.43883, -99.09891 19.441...",09017,Venustiano Carranza,CDMX,urban,17121.0,5529.0,15566.0,4878.0
1764,884995b94dfffff,"POLYGON ((-99.07415 19.42551, -99.06963 19.428...",09017,Venustiano Carranza,CDMX,urban,1670.0,465.0,1681.0,510.0
1765,884995bb33fffff,"POLYGON ((-99.10760 19.43076, -99.10307 19.433...",09017,Venustiano Carranza,CDMX,urban,12975.0,4382.0,12001.0,3753.0
